## Task 2: Lookalike Model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

C:\Users\chott\AppData\Local\Temp\ipykernel_22312\1075886533.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
customer = pd.read_csv("Customers.csv")
product = pd.read_csv("Products.csv")
transaction = pd.read_csv("Transactions.csv")


In [6]:
customer_columns = customer.columns.tolist()
product_columns = product.columns.tolist()
transaction_columns = transaction.columns.tolist()

print("Customer Columns:", customer_columns)
print("Product Columns:", product_columns)
print("Transaction Columns:", transaction_columns)

Customer Columns: ['CustomerID', 'CustomerName', 'Region', 'SignupDate']
Product Columns: ['ProductID', 'ProductName', 'Category', 'Price']
Transaction Columns: ['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate', 'Quantity', 'TotalValue', 'Price']


In [7]:
# Merge transaction and product dataframes to get category information in transactions
transaction_product = pd.merge(transaction, product, on='ProductID')

# Create a pivot table to count the number of purchases per category for each customer
category_pivot = transaction_product.pivot_table(index='CustomerID', columns='Category', values='Quantity', aggfunc='sum', fill_value=0)

# Calculate total spending by each customer
total_spending = transaction.groupby('CustomerID')['TotalValue'].sum()

# Merge the customer dataframe with the category pivot table and total spending
look_alike = pd.merge(customer, category_pivot, on='CustomerID', how='left')
look_alike = pd.merge(look_alike, total_spending, on='CustomerID', how='left')

# Rename the 'TotalValue' column to 'TotalSpending'
look_alike.rename(columns={'TotalValue': 'TotalSpending'}, inplace=True)

In [8]:
look_alike.head()

,CustomerID,CustomerName,Region,SignupDate,Books,Clothing,Electronics,Home Decor,TotalSpending
0,C0001,Lawrence Carroll,South America,2022-07-10,2.0,0.0,7.0,3.0,3354.52
1,C0002,Elizabeth Lutz,Asia,2022-02-13,0.0,4.0,0.0,6.0,1862.74
2,C0003,Michael Rivera,South America,2024-03-07,0.0,4.0,4.0,6.0,2725.38
3,C0004,Kathleen Rodriguez,South America,2022-10-09,8.0,0.0,6.0,9.0,5354.88
4,C0005,Laura Weber,Asia,2022-08-15,0.0,0.0,4.0,3.0,2034.24


In [13]:
look_alike.drop(columns=['CustomerName', 'SignupDate'], inplace=True)
look_alike.head()

,Region,Books,Clothing,Electronics,Home Decor,TotalSpending
CustomerID,,,,,,
C0001,South America,2.0,0.0,7.0,3.0,3354.52
C0002,Asia,0.0,4.0,0.0,6.0,1862.74
C0003,South America,0.0,4.0,4.0,6.0,2725.38
C0004,South America,8.0,0.0,6.0,9.0,5354.88
C0005,Asia,0.0,0.0,4.0,3.0,2034.24


In [15]:
look_alike.drop(columns=['Region'], inplace=True)
look_alike.head()

,Books,Clothing,Electronics,Home Decor,TotalSpending
CustomerID,,,,,
C0001,2.0,0.0,7.0,3.0,3354.52
C0002,0.0,4.0,0.0,6.0,1862.74
C0003,0.0,4.0,4.0,6.0,2725.38
C0004,8.0,0.0,6.0,9.0,5354.88
C0005,0.0,0.0,4.0,3.0,2034.24


In [16]:

data = look_alike

# Set CustomerID as the index if it's not already
if 'CustomerID' in data.columns:
    data.set_index('CustomerID', inplace=True)


# Fill missing values with 0
data.fillna(0, inplace=True)
# Compute cosine similarity matrix without normalizing the data
similarity_matrix = cosine_similarity(data)

# Create a DataFrame for the similarity matrix
similarity_df = pd.DataFrame(
    similarity_matrix, 
    index=data.index, 
    columns=data.index
)

# Round values to 10 decimal places
similarity_df = similarity_df.round(10)

similarity_df

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,0.999993,0.999998,0.999999,1.000000,0.999997,0.999999,0.999998,0.999996,0.999980,...,0.999996,0.999999,0.999993,0.999999,0.999997,0.999997,0.999999,0.999997,0.999997,0.999997
C0002,0.999993,1.000000,0.999998,0.999995,0.999994,0.999995,0.999995,0.999998,0.999994,0.999987,...,0.999988,0.999991,0.999986,0.999996,0.999999,0.999996,0.999991,0.999994,0.999997,0.999997
C0003,0.999998,0.999998,1.000000,0.999998,0.999999,0.999997,0.999999,1.000000,0.999997,0.999986,...,0.999992,0.999996,0.999989,0.999999,1.000000,0.999998,0.999997,0.999997,0.999999,0.999998
C0004,0.999999,0.999995,0.999998,1.000000,0.999999,0.999998,0.999999,0.999997,0.999995,0.999981,...,0.999997,0.999998,0.999996,0.999999,0.999998,0.999998,0.999997,0.999995,0.999998,0.999998
C0005,1.000000,0.999994,0.999999,0.999999,1.000000,0.999997,1.000000,0.999998,0.999996,0.999979,...,0.999994,0.999998,0.999990,0.999999,0.999998,0.999997,0.999999,0.999996,0.999999,0.999997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C0196,0.999997,0.999996,0.999998,0.999998,0.999997,1.000000,0.999998,0.999999,0.999998,0.999987,...,0.999996,0.999997,0.999995,0.999999,0.999999,1.000000,0.999995,0.999998,0.999996,1.000000
C0197,0.999999,0.999991,0.999997,0.999997,0.999999,0.999994,0.999999,0.999996,0.999994,0.999976,...,0.999991,0.999997,0.999987,0.999997,0.999996,0.999995,1.000000,0.999994,0.999998,0.999994
C0198,0.999997,0.999994,0.999997,0.999995,0.999996,0.999998,0.999996,0.999999,1.000000,0.999992,...,0.999993,0.999997,0.999990,0.999998,0.999997,0.999998,0.999994,1.000000,0.999993,0.999999


### The results are not very impressive so we try to normalize as well

In [17]:

data = look_alike

# Set CustomerID as the index if it's not already
if 'CustomerID' in data.columns:
    data.set_index('CustomerID', inplace=True)

# Normalize numerical data for better similarity calculation
data_normalized = (data - data.mean()) / data.std()
data_normalized.fillna(0, inplace=True)
# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(data_normalized)

# Create a DataFrame for the similarity matrix
similarity_df = pd.DataFrame(
    similarity_matrix, 
    index=data.index, 
    columns=data.index
)

similarity_df


CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,-0.340003,0.194331,0.253345,0.658482,-0.892454,0.683035,-0.147544,0.018400,-0.657242,...,-0.078482,0.466566,-0.284041,0.482081,-0.084161,-0.664985,0.834028,0.018226,0.430958,-0.821895
C0002,-0.340003,1.000000,0.721780,-0.356636,0.364724,0.063209,0.399915,0.292228,0.474194,0.376894,...,-0.579972,-0.229192,-0.459290,-0.097522,0.642170,0.380505,0.157825,0.473016,0.652582,0.170839
C0003,0.194331,0.721780,1.000000,-0.086327,0.469954,-0.492528,0.571292,0.670441,0.152238,0.091635,...,-0.905821,-0.286009,-0.893233,0.526988,0.847886,0.081135,0.478970,0.150859,0.690407,-0.000537
C0004,0.253345,-0.356636,-0.086327,1.000000,-0.282057,-0.049431,-0.100864,0.003091,-0.838317,-0.790479,...,0.154705,-0.481474,0.161558,0.568940,0.231447,0.113999,-0.149526,-0.839865,0.022292,-0.077742
C0005,0.658482,0.364724,0.469954,-0.282057,1.000000,-0.708672,0.965183,-0.255006,0.667619,-0.241955,...,-0.242075,0.598577,-0.365899,0.024035,0.055732,-0.457528,0.943162,0.667289,0.841175,-0.790884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C0196,-0.664985,0.380505,0.081135,0.113999,-0.457528,0.742887,-0.298511,0.295025,-0.311008,0.112276,...,-0.288242,-0.790288,-0.093888,0.114390,0.521865,1.000000,-0.648096,-0.307817,-0.091360,0.750990
C0197,0.834028,0.157825,0.478970,-0.149526,0.943162,-0.900636,0.911320,-0.133969,0.494306,-0.320330,...,-0.261379,0.602730,-0.430716,0.210696,0.038138,-0.648096,1.000000,0.493268,0.738846,-0.842910
C0198,0.018226,0.473016,0.150859,-0.839865,0.667289,-0.174067,0.491140,-0.342539,0.999977,0.454345,...,-0.009364,0.705517,-0.049305,-0.627483,-0.267864,-0.307817,0.493268,1.000000,0.407108,-0.376712


### Calculating the simelarity values

In [18]:
# Create a dictionary to store the lookalikes
lookalike_dict = {}

# Iterate through the first 20 customers
for customer_id in similarity_df.index[:20]:
    # Get the similarity scores for the current customer
    similarity_scores = similarity_df.loc[customer_id]
    
    # Sort the similarity scores in descending order and get the top 3 lookalikes
    top_3_lookalikes = similarity_scores.sort_values(ascending=False).index[1:4]
    top_3_scores = similarity_scores.sort_values(ascending=False).values[1:4]
    
    # Store the lookalikes and their scores in the dictionary
    lookalike_dict[customer_id] = list(zip(top_3_lookalikes, top_3_scores))

# Convert the dictionary to a DataFrame
lookalike_df = pd.DataFrame.from_dict(lookalike_dict, orient='index', columns=['Lookalike1', 'Lookalike2', 'Lookalike3'])

# Save the DataFrame to a CSV file
lookalike_df.to_csv('Lookalike.csv', index_label='CustomerID')